In [3]:
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.dataviz import plot_raster
from ephysvibe.trials import firing_rate
from scipy import stats
import pandas as pd
import glob
import os

In [1]:
path='//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/pfc/neurons/2023-03-09_10-35-09_Riesling_pfc_e1_r1_mua23_neu.h5'

In [4]:
neu = NeuronData.from_python_hdf5(path)

## Neutral index

In [ ]:
for inout in ["in", "out"]:
    mask = getattr(neu, "mask_son_" + inout)
    sp_son = getattr(neu, "sp_son_" + inout)
    time_before_son = getattr(neu, "time_before_son_" + inout)

    idx_start_sample = time_before_son + start_sample
    idx_end_sample = time_before_son + end_sample

    fr_son = firing_rate.moving_average(data=sp_son, win=100, step=1)[
        :, idx_start_sample:idx_end_sample
    ]

    sp_t1on = getattr(neu, "sp_t1on_" + inout)
    time_before_t1on = getattr(neu, "time_before_t1on_" + inout)

    idx_start_test = time_before_t1on + start_test
    idx_end_test = time_before_t1on + end_test

    fr_ton = firing_rate.moving_average(data=sp_t1on, win=100, step=1)[
        :, idx_start_test:idx_end_test
    ]

    fr = np.concatenate([fr_son, fr_ton], axis=1)
    sample_id = neu.sample_id[mask]
    fr_samples = select_trials.get_sp_by_sample(fr, sample_id, samples)
    # check trials fr
    for isamp in fr_samples.keys():
        if ~np.all((np.isnan(fr_samples[isamp]))):
            masktr = check_trials(fr_samples[isamp], cerotr, percentile)
            fr_samples[isamp] = fr_samples[isamp][masktr]

    o1 = np.concatenate((fr_samples["11"], fr_samples["15"]))
    o5 = np.concatenate((fr_samples["51"], fr_samples["55"]))
    c1 = np.concatenate((fr_samples["11"], fr_samples["51"]))
    c5 = np.concatenate((fr_samples["15"], fr_samples["55"]))
    sample = np.concatenate(
        (fr_samples["11"], fr_samples["15"], fr_samples["51"], fr_samples["55"])
    )
    inout_nn_fr.append(sample)
    n0 = fr_samples["0"]
    inout_n0_fr.append(n0)

orient_lat, orient_score, orient_p = smetrics.get_selectivity(
    o1, o5, win=75, scores=True
)

## Spatial index

In [5]:
align_sp, alig_mask = neu.align_on(
    select_block=2, event="target_on", time_before=400, error_type=0
)